In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import metrics
import dill as pickle

In [2]:
df_products= pd.read_csv("newUpdated.csv")

In [3]:
df_products.head()

,name,image,description,brand,category,price,countInStock,rating,numReviews
0,Airpods Wireless Bluetooth Headphones,/images/airpods.jpg,Bluetooth technology lets you connect it with ...,Apple,Electronics,89.99,10,4.5,12
1,iPhone 11 Pro 256GB Memory,/images/phone.jpg,Introducing the iPhone 11 Pro. A transformativ...,Apple,Electronics,599.99,7,4.0,8
2,Cannon EOS 80D DSLR Camera,/images/camera.jpg,"Characterized by versatile imaging specs, the ...",Cannon,Electronics,929.99,5,3.0,12
3,Sony Playstation 4 Pro White Version,/images/playstation.jpg,The ultimate home entertainment center starts ...,Sony,Electronics,399.99,11,5.0,12
4,Logitech G-Series Gaming Mouse,/images/mouse.jpg,Get a better handle on your games with this Lo...,Logitech,Electronics,49.99,7,3.5,10


In [4]:
df_products['description']

0     Bluetooth technology lets you connect it with ...
1     Introducing the iPhone 11 Pro. A transformativ...
2     Characterized by versatile imaging specs, the ...
3     The ultimate home entertainment center starts ...
4     Get a better handle on your games with this Lo...
                            ...                        
95    Rapoo H6020 Bluetooth stereo headset with 12 h...
96    Smooth optically reflective material, Bottom m...
97    21.5” Full HD monitor with 1ms (GTG) quick res...
98    Acer ED323QUR Gaming Monitor 32" WQHD LED 144H...
99    2-inch WQHD (2560 x 1440) Fast IPS gaming moni...
Name: description, Length: 100, dtype: object

In [5]:
tfidf = TfidfVectorizer(stop_words="english")
df_products['description'] = df_products['description'].fillna("")
tfidf_matrix = tfidf.fit_transform(df_products['description'])

In [6]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [7]:
indices = pd.Series(df_products.index, index=df_products['name']).drop_duplicates()

In [11]:
indices['Apple iPhone 13']

7

In [13]:
  def get_recommendations(name, cosine_sim = cosine_sim):
      idx = indices[name]
      sim_scores = enumerate(cosine_sim[idx])
      sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
      sim_scores = sim_scores[1:7]
    
      sim_index = [i[0] for i in sim_scores]
  
      y = df_products['name'].iloc[sim_index]
      return y

      
      

  get_recommendations('Apple iPhone 13')  

15               Sony Alpha 7 III
14         Apple iPad Pro 5th Gen
21         Apple MacBook Pro 2021
17    Samsung Galaxy S21 Ultra 5G
18        Microsoft Surface Pro 8
16        LG CX Series 4K OLED TV
Name: name, dtype: object

In [14]:
model = {
    'tfidf': tfidf,
    'cosine_sim': cosine_sim,
    'indices': indices,
    'get_recommendations': get_recommendations,
    'df_products': df_products
}
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)